In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn import svm
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,accuracy_score
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier

In [2]:
df = pd.read_csv("employee.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
df.pop("EmployeeCount")
df.pop("EmployeeNumber")
df.pop("Over18")
df.pop("StandardHours")
y = df["Attrition"].values
x =  df.pop("Attrition").values
bn = LabelBinarizer()
y_train = bn.fit_transform(y)

In [4]:
df.shape

(1470, 30)

In [5]:
df.dtypes

Age                          int64
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole           int64
YearsSinceLastPromot

In [6]:
df.isna().sum()

Age                         0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
dtype: int64

In [7]:
df.select_dtypes(["object"])

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,OverTime
0,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Yes
1,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,No
2,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Yes
3,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Yes
4,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,No
...,...,...,...,...,...,...,...
1465,Travel_Frequently,Research & Development,Medical,Male,Laboratory Technician,Married,No
1466,Travel_Rarely,Research & Development,Medical,Male,Healthcare Representative,Married,No
1467,Travel_Rarely,Research & Development,Life Sciences,Male,Manufacturing Director,Married,Yes
1468,Travel_Frequently,Sales,Medical,Male,Sales Executive,Married,No


In [8]:
ind_BusinessTravel = pd.get_dummies(df["BusinessTravel"],drop_first=True)
ind_Department = pd.get_dummies(df["Department"],drop_first=True)
ind_EducationField = pd.get_dummies(df["EducationField"],drop_first=True)
ind_Gender = pd.get_dummies(df["Gender"],drop_first=True)
ind_JobRole = pd.get_dummies(df["JobRole"],drop_first=True)
ind_MaritalStatus = pd.get_dummies(df["MaritalStatus"],drop_first=True)
ind_OverTime = pd.get_dummies(df["OverTime"],drop_first=True)

In [9]:
df1 = pd.concat([ind_BusinessTravel,ind_Department,ind_EducationField,
           ind_Gender,ind_JobRole,ind_MaritalStatus,ind_OverTime,df.select_dtypes(["int64"])],axis=1)

In [10]:
df1.head()

,Travel_Frequently,Travel_Rarely,Research & Development,Sales,Life Sciences,Marketing,Medical,Other,Technical Degree,Male,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,1,0,1,1,0,0,0,0,0,...,3,1,0,8,0,1,6,4,0,5
1,1,0,1,0,1,0,0,0,0,1,...,4,4,1,10,3,3,10,7,1,7
2,0,1,1,0,0,0,0,1,0,1,...,3,2,0,7,3,3,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,3,3,0,8,3,3,8,7,3,0
4,0,1,1,0,0,0,1,0,0,1,...,3,4,1,6,3,3,2,2,2,2


In [11]:
x = df1.values
x = StandardScaler().fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x,y_train,test_size=0.1,random_state=42)

In [12]:
def Print_result(clf,x_train,x_test,y_train,y_test,train=True):
    """
    print the classification report, confusion matrix and accuracy score
    """
    if train:
        """
        Training Result
        """
        print("Train Result...\n")
        print("Accuracy:\n{0:.4f}".format(accuracy_score(y_train,clf.predict(x_train))))
        print("classification report:\n{}".format(classification_report(y_train,clf.predict(x_train))))
        print("confusion matrix:\n{}".format(confusion_matrix(y_train,clf.predict(x_train))))
        
        cross = cross_val_score(clf,x_train,y_train,cv=15,scoring="accuracy")
        print("Average_score:\n{0:.4f}".format(cross.mean()))
        print("Accuracy_std:\n{0:.4f}".format(cross.std()))
        
    if train==False:
        """
        Testing Result
        """
        print("Test Result...\n")
        print("Accuracy:\n{0:.4f}".format(accuracy_score(y_test,clf.predict(x_test))))
        print("classification report:\n{}".format(classification_report(y_test,clf.predict(x_test))))
        print("confusion matrix:\n{}".format(confusion_matrix(y_test,clf.predict(x_test))))
        
        cross = cross_val_score(clf,x_test,y_test,cv=10,scoring="accuracy")
        print("Average_score:\n{0:.4f}".format(cross.mean()))
        print("Accuracy_std:\n{0:.4f}".format(cross.std()))

In [18]:
def Random_forest(x_train,y_train,train=True):
    rd_clf = RandomForestClassifier(random_state=42)
    bag =BaggingClassifier(base_estimator=rd_clf,n_estimators=100,max_samples=1.0,
                           max_features=1.0,n_jobs=-1,oob_score=True,random_state=42,bootstrap=True)
    bag.fit(x_train,y_train)
    rslt1 = Print_result(bag,x_train,x_test,y_train,y_test,train=True)
    rslt2 = Print_result(bag,x_train,x_test,y_train,y_test,train=False)
    return rslt1,rslt2 
    
def Decission_tree(x_train,y_train,train=True):
    de_clf = RandomForestClassifier()
    bag_clf =BaggingClassifier(base_estimator=de_clf,n_estimators=100,max_samples=1.0,
                           max_features=1.0,n_jobs=-1,oob_score=True,random_state=42,bootstrap=True)
    bag_clf.fit(x_train,y_train)
    rslt1 = Print_result(bag_clf,x_train,x_test,y_train,y_test,train=True)
    rslt2 = Print_result(bag_clf,x_train,x_test,y_train,y_test,train=False)
    print("")
    return rslt1,rslt2
    
def xgboost_(x_train,y_train,train=True):
    xgb_clf = xgb.XGBClassifier(max_depth=3,learning_rate=0.1,n_jobs=-1,gamma=0.1,n_estimators=500)
    bag_clf =BaggingClassifier(base_estimator=xgb_clf,n_estimators=100,max_samples=1.0,
                           max_features=1.0,n_jobs=-1,oob_score=True,random_state=42,bootstrap=True)
    bag_clf.fit(x_train,y_train)
    rslt1 = Print_result(bag_clf,x_train,x_test,y_train,y_test,train=True)
    rslt2 = Print_result(bag_clf,x_train,x_test,y_train,y_test,train=False)
    return rslt1,rslt2

def suport_vector(x_train,y_train,train=True):
    clf = svm.SVC(kernel="rbf",C=1.0,gamma=0.1)
    paras = {"kernel":["linear","hinge","rbf"],
            "C":[1.0,0.1,1.00],
            "gamma":[0.1,0.11,0.01,1.0,"scale"],
             "degree":[2,3],
            "shrinking":[True,False],
            "probability":[True,False]
        }
    grid_svm = GridSearchCV(clf,paras,n_jobs=-1,verbose=1,cv=5)
    grid_svm.fit(x_train,y_train)
    rslt1 = Print_result(grid_svm,x_train,x_test,y_train,y_test,train=True)
    rslt2 = Print_result(grid_svm,x_train,x_test,y_train,y_test,train=False)
    return rslt1,rslt2
                         
def SGB_clf(x_train,y_train,train=True):
    clf = SGDClassifier(random_state=3,loss='modified_huber',penalty='l2',alpha=0.001,
                l1_ratio=0.15, fit_intercept=True,max_iter=200,
                tol=0.001,epsilon=0.011,n_jobs=-1,
                learning_rate='optimal',eta0=0.01,power_t=0.15,
                early_stopping=False,validation_fraction=0.001,
                n_iter_no_change=1,warm_start=True)
                 
    boost_clf = AdaBoostClassifier(base_estimator=clf,n_estimators=50,
                            learning_rate=1.0)

    paras = {"n_estimators":[50,500,100],
        "learning_rate":[0.1,1.0,0.01]}

    grid_boost = GridSearchCV(boost_clf,paras,n_jobs=-1,verbose=1,cv=5)
    grid_boost.fit(x_train,y_train)     
    rslt1 = Print_result(grid_boost,x_train,x_test,y_train,y_test,train=True)
    rslt2 = Print_result(grid_boost,x_train,x_test,y_train,y_test,train=False)
    return rslt1,rslt2

In [19]:
suport_vector(x_train,y_train,train=True)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 25.7min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 25.7min finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train Result...

Accuracy:
0.8987
classification report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1109
           1       0.85      0.45      0.59       214

    accuracy                           0.90      1323
   macro avg       0.88      0.72      0.77      1323
weighted avg       0.89      0.90      0.89      1323

confusion matrix:
[[1092   17]
 [ 117   97]]
Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 21.8min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed: 21.8min finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


...

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Done 540 tasks      | elapsed:    5.0s


Average_score:
0.8362
Accuracy_std:
0.0565


[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:   12.2s finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(None, None)

In [20]:
Decission_tree(x_train,y_train,train=True)

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train Result...

Accuracy:
0.9652
classification report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1109
           1       1.00      0.79      0.88       214

    accuracy                           0.97      1323
   macro avg       0.98      0.89      0.93      1323
weighted avg       0.97      0.97      0.96      1323

confusion matrix:
[[1109    0]
 [  46  168]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning

Average_score:
0.8496
Accuracy_std:
0.0170
Test Result...

Accuracy:
0.8707
classification report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       124
           1       1.00      0.17      0.30        23

    accuracy                           0.87       147
   macro avg       0.93      0.59      0.61       147
weighted avg       0.89      0.87      0.83       147

confusion matrix:
[[124   0]
 [ 19   4]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning

Average_score:
0.8438
Accuracy_std:
0.0289


(None, None)

In [21]:
Random_forest(x_train,y_train,train=True)

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train Result...

Accuracy:
0.9652
classification report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1109
           1       1.00      0.79      0.88       214

    accuracy                           0.97      1323
   macro avg       0.98      0.89      0.93      1323
weighted avg       0.97      0.97      0.96      1323

confusion matrix:
[[1109    0]
 [  46  168]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning

Average_score:
0.8496
Accuracy_std:
0.0170
Test Result...

Accuracy:
0.8707
classification report:
              precision    recall  f1-score   support

           0       0.87      1.00      0.93       124
           1       1.00      0.17      0.30        23

    accuracy                           0.87       147
   macro avg       0.93      0.59      0.61       147
weighted avg       0.89      0.87      0.83       147

confusion matrix:
[[124   0]
 [ 19   4]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning

Average_score:
0.8438
Accuracy_std:
0.0289


(None, None)

In [22]:
xgboost_(x_train,y_train,train=True)

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train Result...

Accuracy:
0.9932
classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1109
           1       1.00      0.96      0.98       214

    accuracy                           0.99      1323
   macro avg       1.00      0.98      0.99      1323
weighted avg       0.99      0.99      0.99      1323

confusion matrix:
[[1109    0]
 [   9  205]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning

Average_score:
0.8715
Accuracy_std:
0.0225
Test Result...

Accuracy:
0.8980
classification report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94       124
           1       0.83      0.43      0.57        23

    accuracy                           0.90       147
   macro avg       0.87      0.71      0.76       147
weighted avg       0.89      0.90      0.88       147

confusion matrix:
[[122   2]
 [ 13  10]]


c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\_bagging.py:645: DataConversionWarning

Average_score:
0.8705
Accuracy_std:
0.0367


(None, None)

In [23]:
SGB_clf(x_train,y_train,train=True)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.3min finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train Result...

Accuracy:
0.8745
classification report:
              precision    recall  f1-score   support

           0       0.88      0.98      0.93      1109
           1       0.79      0.31      0.44       214

    accuracy                           0.87      1323
   macro avg       0.83      0.65      0.69      1323
weighted avg       0.87      0.87      0.85      1323

confusion matrix:
[[1091   18]
 [ 148   66]]
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  1.1min finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   31.1s finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   28.5s finished
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Average_score:
0.8290
Accuracy_std:
0.0651


(None, None)

plt.hist(df.names,df.names2,bin=35)
